In [1]:
import smbus2
import time
import json
import paho.mqtt.client as mqtt

#create the i2c bus
bus = smbus2.SMBus(1)

def on_connect(client, userdata, flags, rc):
    print("Connected with result code: " + str(rc))

client = mqtt.Client()
client.on_connect = on_connect
#set private key 
#(ca_certs=BROKER_CERT,certfile=CLIENT_CERT,keyfile=PRIVATE_KEY,tls_version=ssl.PROTOCOL_TLSv1_2)
#client.tls_set(ca_certs="mosquitto.org.crt", certfile="client.crt",keyfile='client.key')
#(BROKER_ADDRESS,port=N)
client.connect("broker.mqttdashboard.com",port=1883)

while True:
    meas_temp= smbus2.i2c_msg.write(0x40,[0xf3])
    bus.i2c_rdwr(meas_temp)

    #wait for measurement
    time.sleep(0.1)

    #send the read temperature command and read 2 bytes of data
    read_result= smbus2.i2c_msg.read(0x40,2)
    bus.i2c_rdwr(read_result)

    #convert the result to INT type
    temp = int.from_bytes(read_result.buf[0]+read_result.buf[1],'big')

    #package into JSON
    sensor_data = {'temp':temp,'name':'hexfuture','time':time.time()}
    payload = json.dumps(sensor_data)
    #pulish message
    MSG_INFO = client.publish("IC.embedded/hexfuture/test",payload,qos=2)
    mqtt.error_string(MSG_INFO.rc)
    print(payload)
    time.sleep(60)




{"temp": 26692, "name": "hexfuture", "time": 1644404148.3865702}


KeyboardInterrupt: 

In [21]:
import smbus2
bus = smbus2.SMBus(1)

meas_temp= smbus2.i2c_msg.write(0x40,[0xf3])
bus.i2c_rdwr(meas_temp)

#wait for measurement
time.sleep(0.1)

#send the read temperature command and read 2 bytes of data
read_result= smbus2.i2c_msg.read(0x40,2)
bus.i2c_rdwr(read_result)

#convert the result to INT type
temp = int.from_bytes(read_result.buf[0]+read_result.buf[1],'big')

print (temp)


26588


In [29]:
import smbus2
bus = smbus2.SMBus(1)


 
time.sleep(0.3)
bus.write_byte(0x40, 0xF3)
time.sleep(0.3)
 
# SI7021 address, 0x40 Read data 2 bytes, Temperature
data0 = bus.read_byte(0x40)
data1 = bus.read_byte(0x40)

# Convert the data
humidity = ((data0 * 256 + data1) * 125 / 65536.0) - 6

# Convert the data and output it
celsTemp = ((data0 * 256 + data1) * 175.72 / 65536.0) - 46.85
fahrTemp = celsTemp * 1.8 + 32
 
print ("Relative Humidity is : %.2f %%" %humidity)
print ("Temperature in Celsius is : %.2f C" %celsTemp)
print ("Temperature in Fahrenheit is : %.2f F" %fahrTemp)


Relative Humidity is : 45.96 %
Temperature in Celsius is : 26.19 C
Temperature in Fahrenheit is : 79.15 F
